In [12]:
import numpy as np
import tempfile
from py_irt.io import write_jsonlines
from py_irt.dataset import Dataset
from py_irt.config import IrtConfig
from py_irt.training import IrtModelTrainer
import joblib as jbl

def fit_pyirt(X, model_type="2pl", epochs=1000, lr=0.05, device="cpu"):
    X = np.asarray(X, float)
    n, k = X.shape
    ii, jj = np.where(np.isfinite(X))

    rows = [{"subject_id": str(i), "responses": {}} for i in range(n)]
    for i, j in zip(ii.tolist(), jj.tolist()):
        rows[i]["responses"][str(j)] = int(X[i, j])

    with tempfile.NamedTemporaryFile("w", suffix=".jsonlines", delete=False) as f:
        write_jsonlines(f.name, rows)
        dataset = Dataset.from_jsonlines(f.name)

    trainer = IrtModelTrainer(config=IrtConfig(model_type=model_type, lr=lr), data_path=None, dataset=dataset)
    trainer.train(epochs=epochs, device=device)
    return trainer

In [13]:
def predict_proba_matrix(model, n, k):
    subjects = np.repeat(np.arange(n), k)   # ints
    items    = np.tile(np.arange(k), n)     # ints
    p = np.asarray(model.predict(subjects.tolist(), items.tolist()), dtype=float)
    return p.reshape(n, k)

In [14]:
data = jbl.load("Data/mmlu_pro.jbl")

In [15]:
trainer = fit_pyirt(data)

torch.Size([5390336]) torch.Size([5390336])


Output()

In [21]:
np.array([trainer.irt_model.export()["diff"],trainer.irt_model.export()["disc"]]).T

(12032, 2)

[0.2311772257089615,
 0.014537263661623001,
 0.4831485450267792,
 0.5157025456428528,
 0.5422438383102417,
 0.5659587979316711,
 0.56027752161026,
 0.47261860966682434,
 0.290729820728302,
 0.10235224664211273,
 0.4517870545387268,
 0.4880375266075134,
 0.030811332166194916,
 0.28386804461479187,
 0.014647201634943485,
 0.14689330756664276,
 0.32164889574050903,
 0.16614864766597748,
 0.07612314075231552,
 0.1810692846775055,
 0.519173264503479,
 0.0971233919262886,
 0.5688953399658203,
 0.010669911280274391,
 0.46144899725914,
 0.3353068232536316,
 0.5572033524513245,
 0.012946700677275658,
 0.3055485188961029,
 0.25102365016937256,
 0.01115708239376545,
 0.2318715900182724,
 0.3258213400840759,
 0.2362728863954544,
 0.2705119550228119,
 0.11452783644199371,
 0.36020973324775696,
 0.14558489620685577,
 0.2408304065465927,
 0.20656825602054596,
 0.3087443709373474,
 0.26666194200515747,
 0.3674166202545166,
 0.16769061982631683,
 0.46537116169929504,
 0.03880651295185089,
 0.0574811622

In [22]:
np.save("Data/irt2p_mmlu_pro",np.array([trainer.irt_model.export()["diff"],trainer.irt_model.export()["disc"]]).T)

In [ ]:
np.save("Data/irt2p_mmlu_pro",trainer.irt_model.export()["diff"])

In [ ]:
((predict_proba_matrix(trainer.irt_model,data.shape[0],data.shape[1]) - data)**2).mean()